In [2]:
from sklearn.metrics import adjusted_rand_score as ARI
from sklearn.metrics import adjusted_mutual_info_score as AMI 
import hypernetx as hnx
import hypernetx.algorithms.hypergraph_modularity as hmod
from collections import Counter
import igraph as ig
import pandas as pd
import numpy as np
import h_louvain as hl
import csv



In [3]:
def load_ABCDH_from_file(filename):
    with open(filename,"r") as f:
        rd = csv.reader(f)
        lines = list(rd)
    Edges = []
    for line in lines:
        Edges.append(list(line))

    HG = hnx.Hypergraph(dict(enumerate(Edges)))
    return HG

In [4]:
HG = load_ABCDH_from_file("datasets/results_300_he.txt")



In [5]:
hL = hl.hLouvain(HG,hmod_type=hmod.linear, 
                                    delta_it = 0.0001, 
                                    delta_phase = 0.0001, 
                                    random_seed = 5673) 

In [6]:
alphas = []
d = 0
c = 0.16
b = 0.8
for i in range(30):
    alphas.append(1-((1-b)**i)*(1-d))
    
print("alphas", alphas)
    
A, q2, alphas_out = hL.h_louvain_community(alphas = alphas, 
                                                change_mode="communities", change_frequency=c)
qH = hL.combined_modularity(A, hmod.linear, 1)

print("alphas_out", alphas_out)

print("partition",A)
print("gH =",qH)

alphas [0.0, 0.09999999999999998, 0.18999999999999995, 0.2709999999999999, 0.3439, 0.40950999999999993, 0.46855899999999995, 0.5217030999999999, 0.5695327899999999, 0.6125795109999999, 0.6513215599, 0.6861894039099999, 0.7175704635189999, 0.7458134171670999, 0.7712320754503899, 0.794108867905351, 0.8146979811148158, 0.8332281830033342, 0.8499053647030008, 0.8649148282327007, 0.8784233454094307, 0.8905810108684876, 0.9015229097816388, 0.9113706188034749, 0.9202335569231275, 0.9282102012308147, 0.9353891811077333, 0.94185026299696, 0.9476652366972639, 0.9528987130275375]
alphas_out [0.0, 0.09999999999999998]
partition [{'228', '78', '222', '229', '72', '288', '12', '18', '109', '210', '251', '221', '116', '266', '88', '151', '173', '247', '143', '296', '152', '95', '46', '223'}, {'257', '138', '164', '23', '178', '146', '157', '289', '267', '86', '107', '30', '98', '184', '214', '208', '273', '115', '279'}, {'154', '189', '71', '68', '87', '268', '34', '90', '121', '92', '110', '280', '2

In [47]:
qH = hL.combined_modularity(A, hmod.linear, 1)
print(qH)
two_section_modularity = hL.combined_modularity(A, hmod.linear, 0)
print(two_section_modularity)

0.5245488853644789
0.45520273851254306


In [37]:
with open("Data/results_300_assign.txt", 'r') as file:
    gt = [int(line) for line in file]
A_gt = [x for x in hmod.dict2part({str(i+1):gt[i] for i in range(len(gt))}) if len(x)>0]
gt_linear = hL.combined_modularity(A_gt, hmod.linear, 1)

print("ground-truth partition")
print(A_gt)
print("qh-gt-linear:",gt_linear)

def getAMI_ARI(HG,gt,A):
    d = hmod.part2dict(A)
    A4ari = [d[str(i+1)] for i in range(len(HG.nodes))]
    return ARI(gt, A4ari), AMI(gt, A4ari)

ground-truth partition
[{'81', '272', '197', '125', '254', '285', '148', '183', '300', '191', '124', '112', '117', '144', '65', '96', '77', '135', '286', '182', '203', '134', '79', '70', '14', '227', '260', '271'}, {'141', '89', '16', '34', '209', '154', '185', '226', '121', '268', '119', '123', '90', '230', '41', '110', '189', '92', '83', '68', '198', '87', '280', '71', '28'}, {'251', '296', '229', '265', '151', '95', '88', '228', '223', '116', '78', '266', '222', '143', '109', '210', '12', '173', '18', '152', '288', '72', '221', '247', '46'}, {'177', '236', '167', '283', '261', '13', '270', '186', '162', '180', '290', '44', '147', '62', '235', '4', '114', '63', '40', '82', '32', '255'}, {'58', '166', '104', '31', '172', '102', '140', '299', '201', '45', '2', '174', '216', '195', '67', '257', '258', '35', '49', '269', '200', '29'}, {'93', '190', '238', '80', '213', '241', '276', '179', '169', '74', '57', '50', '211', '284', '297', '66', '43', '103', '149'}, {'273', '138', '289', '107'

In [38]:
ari, ami = getAMI_ARI(HG,gt,A)
print("ARI =", ari)
print("AMI =", ami)
print("comm =", len(A))
print("comm-gt =", len(A_gt))

ARI = 0.7355924918874397
AMI = 0.8354839400985541
comm = 15
comm-gt = 18
